In [1]:
import uuid
from googleads import adwords
import os
import datetime
auth_file_path = '/home/sharefolder/Docs/Google_Ads_Access/googleads_BL_MMC.yaml'

def upload_and_return_count_in_chunk(service, user_list_id, email_list):
    ids_in_chunk=0
    mutate_members_operation = {
      'operand': {
          'userListId': user_list_id,
          'membersList': email_list
      },
      'operator': 'ADD'
    }
    response = service.mutateMembers([mutate_members_operation])
    if 'userLists' in response:
        for user_list in response['userLists']:
            # print('Uploaded %i items to user list with name "%s" and ID "%d"'
            #     % (len(email_list), user_list['name'], user_list['id']))

            
            ids_in_chunk+=len(email_list)
    return ids_in_chunk
            
def run_the_uploading(client, email_file_path):
  # Initialize appropriate services.
    total_ids_in_seg=0
    user_list_service = client.GetService('AdwordsUserListService', 'v201809')
    file_base_name=os.path.basename(email_file_path).split(".")[0]
    user_list = {
        'xsi_type': 'CrmBasedUserList',
        'name': file_base_name.replace("removed_Q3_QuarterlyControl_09142020_","aud_2020Step_")+"_directly_uploaded",
        'description': 'Aggregated control for 2020Q1 RFM: '+file_base_name,
        # CRM-based user lists can use a membershipLifeSpan of 10000 to indicate
        # unlimited; otherwise normal values apply.
        'membershipLifeSpan': 10000,
        'uploadKeyType': 'CONTACT_INFO'
     }
    # Create an operation to add the user list.
    operations = [{
        'operator': 'ADD',
        'operand': user_list
    }]
    result = user_list_service.mutate(operations)
    user_list_id = result['value'][0]['id']
    
    counter_invalid_email_count=0
    chuncksize_n=3000
    df = pd.read_csv(email_file_path, chunksize=chuncksize_n)
    for i in df:
        mail_col = [k for k in i.columns if 'email' in k.lower()][0]
        new_col = ['email' if h == mail_col else h for h in i.columns.tolist()]
        i.columns = new_col
        emails = i['email']
        
        emails = set([x for x in emails if len(str(x))==64])
        counter_invalid_email_count+=(i.shape[0]-len(emails))
        members = [{'hashedEmail': email} for email in emails]
        ids_in_chunk=upload_and_return_count_in_chunk(user_list_service, user_list_id, members)
        
        total_ids_in_seg+=ids_in_chunk
    print(datetime.datetime.now(),"all %s ids uploaded in the segment "%str(total_ids_in_seg))
    print("%i of it is invalid emails"%counter_invalid_email_count)

In [2]:
import glob
import pandas as pd
import glob
folder_client_segs="/home/jian/Projects/Big_Lots/Analysis/2020_Q3/Audience_for_21mm_Act_Goal/Uploaded_Files/PYD_clients/"
list_files_to_upload_PYD=glob.glob(folder_client_segs+"*.csv")
# list_files_to_upload_PYD=[x for x in list_files_to_upload_PYD if "_C.csv" not in x]
list_files_to_upload_PYD.sort()
list_files_to_upload_PYD

['/home/jian/Projects/Big_Lots/Analysis/2020_Q3/Audience_for_21mm_Act_Goal/Uploaded_Files/PYD_clients/removed_Q3_QuarterlyControl_09142020_PYD_Active_A.csv',
 '/home/jian/Projects/Big_Lots/Analysis/2020_Q3/Audience_for_21mm_Act_Goal/Uploaded_Files/PYD_clients/removed_Q3_QuarterlyControl_09142020_PYD_Active_B.csv',
 '/home/jian/Projects/Big_Lots/Analysis/2020_Q3/Audience_for_21mm_Act_Goal/Uploaded_Files/PYD_clients/removed_Q3_QuarterlyControl_09142020_PYD_Active_C.csv',
 '/home/jian/Projects/Big_Lots/Analysis/2020_Q3/Audience_for_21mm_Act_Goal/Uploaded_Files/PYD_clients/removed_Q3_QuarterlyControl_09142020_PYD_Lapsed_A.csv',
 '/home/jian/Projects/Big_Lots/Analysis/2020_Q3/Audience_for_21mm_Act_Goal/Uploaded_Files/PYD_clients/removed_Q3_QuarterlyControl_09142020_PYD_Lapsed_B.csv',
 '/home/jian/Projects/Big_Lots/Analysis/2020_Q3/Audience_for_21mm_Act_Goal/Uploaded_Files/PYD_clients/removed_Q3_QuarterlyControl_09142020_PYD_Lapsed_C.csv',
 '/home/jian/Projects/Big_Lots/Analysis/2020_Q3/Audi

In [3]:
# check the customersummary_emailhash hashed validation
df=pd.read_csv(list_files_to_upload_PYD[0])
df['len_email']=df['customersummary_emailhash'].apply(len)
df.groupby("len_email")['customersummary_emailhash'].count().to_frame().reset_index()

,len_email,customersummary_emailhash
0,7,49099
1,64,6424845


In [4]:
BL_client = adwords.AdWordsClient.LoadFromStorage(auth_file_path)
for local_seg_file in list_files_to_upload_PYD:
    run_the_uploading(BL_client, local_seg_file)

2020-09-17 01:07:52.453976 all 6424845 ids uploaded in the segment 
49099 of it is invalid emails
2020-09-17 01:28:37.757186 all 6425837 ids uploaded in the segment 
48908 of it is invalid emails
2020-09-17 02:10:47.769048 all 6425035 ids uploaded in the segment 
49302 of it is invalid emails
2020-09-17 02:18:36.158268 all 1174859 ids uploaded in the segment 
0 of it is invalid emails
2020-09-17 02:26:19.466846 all 1174618 ids uploaded in the segment 
0 of it is invalid emails
2020-09-17 02:33:55.729385 all 1174524 ids uploaded in the segment 
0 of it is invalid emails
2020-09-17 02:37:10.687380 all 971779 ids uploaded in the segment 
27 of it is invalid emails
2020-09-17 02:40:17.305852 all 971768 ids uploaded in the segment 
27 of it is invalid emails
2020-09-17 02:43:27.779136 all 971785 ids uploaded in the segment 
17 of it is invalid emails
